In [ ]:
from bs4 import BeautifulSoup as BS
import pandas as pd
import csv
import requests
from tqdm import tqdm
from pandas.io.sql import SQLTable
import webscrape_pd_scrape_patheos as pscrape
import sys
from passlib.hash import pbkdf2_sha256
try:
    from configparser import ConfigParser
    from configparser import SafeConfigParser
except ImportError:
    from ConfigParser import ConfigParser  # ver. < 3.0
    from ConfigParser import SafeConfigParser

In [ ]:
def update_system_status_values(file, section, option, value):
    config = ConfigParser()
    config.read(file)
    cfgfile = open(file, 'w')
    if config.has_section(section) == True:
    #try:
        config.set(section, option, value)
    else:
    #except:
        config.add_section(section)
        config.set(section, option, value)
    config.write(cfgfile)
    cfgfile.close()
    
#update_system_status_values('patheos_blogs_info.ini', 'EVANGELICAL', 'new', '100')

In [ ]:
def exists_config_file(file, section, option):
    config = ConfigParser()
    config.read(file)
    if config.has_option(section, option) == True:
        page_number = int(config.get(section, option))
    else:
        page_number = 1
    return page_number

In [ ]:
url = 'http://www.patheos.com/blogs'
response = requests.get(url)
soup = BS(response.content, 'html.parser')
#soup.prettify

In [ ]:
blog_list = []

for blog in soup.find_all('div', attrs={"class":"related-content clearfix related-content-sm decorated channel-list"}):
    blog_url1 = blog.find('a')
    blog_url2 = blog_url1['href']
    blog_list.append(blog_url2) 

#for i in blog_list:
#    print(i)

In [ ]:
blog_lists = []
    
for i in blog_list:
    split_url = i.rsplit("/")
    blog_name = split_url[3]
    blog_lists.append(blog_name)
    
blog_dict = {i:[] for i in blog_lists}
#print(blog_dict)

#print(blog_dict['buddhist-blogs'])

In [ ]:
#for i in blog_dict:
#    print(i.upper())
#    for v in blog_dict[i]:
#        print("    - " + v)

In [ ]:
#for i in blog_dict:
#    print("https://www.patheos.com/" + i)

In [ ]:
blog_cat_prefix = "https://www.patheos.com/"

while True:
    i = 0    
    for blog_urls in tqdm(blog_dict, desc='Fetch blog urls for each category'):
        #print(blog_urls)
        if blog_urls:
            query_url = blog_cat_prefix + blog_urls
            subsub_blog = requests.get(query_url)
            #if subsub_blog != "<Response [404]>":
            soup2 = BS(subsub_blog.content, 'html.parser')
            for blog in soup2.find_all('div', attrs={"class":"author-info"}):
                #print(blog)
                for blog_url0 in blog.find_all('div', attrs={"class":"title"}):
                    #print(blog_url0)
                    blog_url1 = blog_url0.find('a')
                    blog_url2 = blog_url1['href']
                    blog_dict[blog_urls].append(blog_url2)
                    #print(blog_url2)
            i = i + 1
        else:
            continue
    else:
        break

In [ ]:
def test_404(blog, page_number, increment):
    pn = 0
    return_str = page_number
    continue_on = True
    if page_number < 1:
        pn = increment
    else:
        pn = page_number + increment
    #print("page number: " + str(pn))
    while continue_on == True:
        blog_page_url = blog + "/page/" + str(pn)
    #print(blog_page_url)
        #print("\n" + blog_page_url)
        try:
            url_test = requests.get(blog_page_url)
            if url_test.status_code != 404:
                return_str = pn
                pn = pn + increment
            else:
                continue_on = False
        except:
            return_str = -1
        #print(url_test.status_code)
        
            
    return return_str

In [ ]:
def blog_find_pages(blogname_list, list_name):
    c = 0
    t = 1
    b = 1
    url_list = []
    continue_on = True
    for i in blogname_list:
        #p = 1
        if i != "":
            try:
                continue_on = True
                jump_100 = True
                page_prefix = "/page/"
                blog_name = i.rsplit("/")
                try:
                    p = int(exists_config_file('patheos_blogs_pages.ini', list_name, blog_name[4]))
                except:
                    p = 1

                #trigger_25 = False
                checkpoint = 0
                #print("\n" + blog_name[4].upper())

                try:
                    p1 = test_404(i, p, 500)
                except:
                    continue
                
                try:
                    sys.stdout.write('\r' + list_name.upper() + " | " + blog_name[4] + " | category number: " + str(b) + "/" + str(len(blogname_list)) 
                                     + " | page(" + str(p) + ")")
                except:
                    continue
                if p1 != -1:
                    p2 = test_404(i, p1, 100)
                    p = p1
                    if p2 != -1:
                        p3 = test_404(i, p2, 50)
                        p = p2
                        if p3 != -1:
                            p4 = test_404(i, p3, 25)
                            p = p4
                if p != -1:
                    while continue_on == True:    
                        if checkpoint > 20:
                            update_system_status_values('patheos_blogs_pages.ini', list_name, blog_name[4], str(p))
                        blog_page_url = i + page_prefix + str(p)
                        url_test = requests.get(blog_page_url)
                        if url_test.status_code != 404:
                            url_list.append(blog_page_url)
                            #print(blog_page_url)
                            p = p + 1
                            #t = t + 1
                            checkpoint = checkpoint + 1
                            #c = c + 1
                            continue_on = True
                        else:
                            #e = e + 1
                            p = p - 1
                            continue_on = False

                        sys.stdout.write('\r' + list_name.upper() + " | " + blog_name[4] + " | category number: " + str(b) + "/" + str(len(blogname_list)) 
                                         + " | page(" + str(p) + ")") # | category total: (" + str(t) + ")")
                    t = t + p
                    b = b + 1
                    update_system_status_values('patheos_blogs_pages.ini', list_name, blog_name[4], str(p))
            except:
                continue
            
    #else:
        #return url_list

fail_list = []


In [ ]:
# For in file testing
keep = True
for i in blog_dict:
#for i in blog_dict:
    while keep == True:
        print(i)
        name = i
        keep = False
        for b in blog_dict['progressive-christian-blogs']:
            print(b)
        #new_list = blog_find_pages(blog_dict[i], i)
        blog_find_pages(blog_dict['progressive-christian-blogs'], 'progressive-christian-blogs')

In [ ]:
#for i in final_list:
#    print(i)

In [ ]:
# For in file testing
n = 0

for i in blog_dict:
    print(i, len(blog_dict[i]))
    n = n + len(blog_dict[i])

print("Total: " + str(n))